In [2]:
from datasets import load_dataset

In [ ]:
data = load_dataset("Synthyra/BIOGRID", split="train")
data


In [ ]:
seqs = set()

data = data.shuffle(seed=42)

def process_dataset(ex):
    seq_a, seq_b = ex['SeqA'], ex['SeqB']
    if seq_a in seqs or seq_b in seqs:
        return False
    seqs.add(seq_a)
    seqs.add(seq_b)
    return True

data = data.filter(process_dataset)
data

In [ ]:
data = load_dataset('Synthyra/all_string_pairs_700', split='train')
data

In [ ]:
class UniqueIDFilter:
    def __init__(self):
        self.ids_seen = set()
    
    def __call__(self, ex):
        if ex['score'] < 900:
            return False
        id_a, id_b = ex['pairs'].split('|')
        if id_a in self.ids_seen or id_b in self.ids_seen:
            return False
        self.ids_seen.add(id_a)
        self.ids_seen.add(id_b)
        return True


filter_func = UniqueIDFilter()
data = data.filter(filter_func, num_proc=16)
data

In [ ]:
data = load_dataset('Synthyra/all_string_pairs_900_unique', split='train')
ids = set()
for pair in data['pairs']:
    id_a, id_b = pair.split('|')
    ids.add(id_a)
    ids.add(id_b)
del data
len(ids)

In [ ]:
string_seqs = load_dataset('Synthyra/StringDBSeqsv12', split='train')


def get_relevant_ids(ex):
    id = ex['id']
    if id in ids:
        return True
    return False

string_seqs = string_seqs.filter(get_relevant_ids)
string_seqs

In [ ]:
string_seqs.push_to_hub('Synthyra/StringDBSeqsv12_unique', private=True)

In [3]:
seqs = set(load_dataset('Synthyra/StringDBSeqsv12_unique', split='train')['sequence'])
len(seqs)

README.md:   0%|          | 0.00/336 [00:00<?, ?B/s]

train-00000-of-00016.parquet:   0%|          | 0.00/579M [00:00<?, ?B/s]

train-00001-of-00016.parquet:   0%|          | 0.00/631M [00:00<?, ?B/s]

train-00002-of-00016.parquet:   0%|          | 0.00/576M [00:00<?, ?B/s]

train-00003-of-00016.parquet:   0%|          | 0.00/549M [00:00<?, ?B/s]

...

train-00014-of-00016.parquet:   0%|          | 0.00/469M [00:00<?, ?B/s]

train-00015-of-00016.parquet:   0%|          | 0.00/496M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20019684 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/18 [00:00<?, ?it/s]

19582099

In [4]:
from tqdm.auto import tqdm


av = load_dataset('lhallee/AV_large', split='train', streaming=True)

counter, total = 0, 0

pbar = tqdm(av)
for ex in pbar:
    if ex['sequence'] in seqs:
        counter += 1
    total += 1
    pbar.set_description(f'{counter}/{total}')

counter


README.md:   0%|          | 0.00/335 [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/42 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/42 [00:00<?, ?it/s]

0it [00:00, ?it/s]

KeyboardInterrupt: 